# Import necessary libraries


In [ ]:
import numpy as np
import xarray as xr
import os


# Define the function to calculate Earth's radius at a given latitude


In [ ]:
def earth_radius(lat):
    """
    Calculate the radius of Earth assuming an oblate spheroid defined by WGS84.
    """
    a = 6378137  # semi-major axis in meters
    b = 6356752.3142  # semi-minor axis in meters
    e2 = 1 - (b**2 / a**2)  # square of eccentricity
    
    lat_rad = np.deg2rad(lat)  # convert from degrees to radians
    lat_gc = np.arctan((1 - e2) * np.tan(lat_rad))  # convert from geodetic to geocentric
    
    r = (a * (1 - e2)**0.5) / (1 - (e2 * np.cos(lat_gc)**2))**0.5
    return r


# Define the function to calculate the area of each grid cell


In [ ]:
def area_grid(lat, lon):
    """
    Calculate the area of each grid cell in square meters.
    """
    lon, lat = np.meshgrid(lon, lat)  
    R = earth_radius(lat)
    
    dlat = np.deg2rad(np.gradient(lat, axis=0))
    dlon = np.deg2rad(np.gradient(lon, axis=1))
    
    dy = dlat * R
    dx = dlon * R * np.cos(np.deg2rad(lat))
    
    area = dy * dx  # Area calculation
    return area


# Unique latitudes and longitudes provided


In [ ]:
latitudes = np.array([25.91623037, 26.85863874, 27.80104712, 28.7434555, 29.68586387, 
                      30.62827225, 31.57068063, 32.51308901, 33.45549738, 34.39790576, 
                      35.34031414, 36.28272251, 37.22513089, 38.16753927, 39.10994764, 
                      40.05235602, 40.9947644, 41.93717277, 42.87958115, 43.82198953, 
                      44.76439791, 45.70680628, 46.64921466, 47.59162304, 48.53403141, 
                      49.47643979])
longitudes = np.array([235., 236.25, 237.5, 238.75, 240., 241.25, 242.5, 243.75, 245., 246.25,
                       247.5, 248.75, 250., 251.25, 252.5, 253.75, 255., 256.25, 257.5, 258.75,
                       260., 261.25, 262.5, 263.75, 265., 266.25, 267.5, 268.75, 270., 271.25,
                       272.5, 273.75, 275., 276.25, 277.5, 278.75, 280., 281.25, 282.5, 283.75,
                       285., 286.25, 287.5, 288.75, 290., 291.25, 292.5, 293.75, 295.])


# Calculate the area of each grid cell and normalization factors


In [ ]:
# Calculate the area of each grid cell
area = area_grid(latitudes, longitudes)
total_area = np.sum(area)

# Compute the normalization factor for each cell relative to the average area
average_area_per_cell = total_area / area.size
normalized_areas = area / average_area_per_cell


# Define file paths and variable names for each metric


In [ ]:
# File paths and variable names for each metric
file_info = {
    'daysbelow0': {
        '1991_2020': '/home/shawn_preston/daysbelow0ensemble/tavgdaysbelow0_1991_2020.nc',
        '2031_2060': '/home/shawn_preston/daysbelow0ensemble/tavgdaysbelow0_2031_2060.nc',
        'var_name_1991_2020': 'days_below_0_ensemble_1991_2020',
        'var_name_2031_2060': 'days_below_0_ensemble_2031_2060'
    },
    'frost': {
        '1991_2020': '/home/shawn_preston/lastdayfrostensemble/last_frost_1991_2020.nc',
        '2031_2060': '/home/shawn_preston/lastdayfrostensemble/last_frost_2031_2060.nc',
        'var_name_1991_2020': 'last_frost_dates_ensemble',
        'var_name_2031_2060': 'last_frost_dates_ensemble'
    },
    'gddapr': {
        '1991_2020': '/home/shawn_preston/gddcesm2janapr/GDD_1991_2020.nc',
        '2031_2060': '/home/shawn_preston/gddcesm2janapr/GDD_2031_2060.nc',
        'var_name_1991_2020': 'gdd_ensemble_1991_2020',
        'var_name_2031_2060': 'gdd_ensemble_2031_2060'
    },
    'gddsep': {
        '1991_2020': '/home/shawn_preston/gddcdesm2jansept/GDD_1991_2020.nc',
        '2031_2060': '/home/shawn_preston/gddcdesm2jansept/GDD_2031_2060.nc',
        'var_name_1991_2020': 'gdd_ensemble_1991_2020',
        'var_name_2031_2060': 'gdd_ensemble_2031_2060'
    },
    'tmax': {
        '1991_2020': '/home/shawn_preston/tmax34ensemble/tmax1591_20.nc',
        '2031_2060': '/home/shawn_preston/tmax34ensemble/tmax1531_60.nc',
        'var_name_1991_2020': 'heat_ensemble_1991_2020',
        'var_name_2031_2060': 'heat_ensemble_2031_2060'
    },
    'tmin': {
        '1991_2020': '/home/shawn_preston/tmin15ensemble/tmin1591_20.nc',
        '2031_2060': '/home/shawn_preston/tmin15ensemble/tmin1531_60.nc',
        'var_name_1991_2020': 'night_ensemble_1991_2020',
        'var_name_2031_2060': 'night_ensemble_2031_2060'
    }
}


# Define the base path for saving new files and create the directory if it does not exist


In [ ]:
# Base path for saving new files
base_path = "/home/shawn_preston/weightedcesm2"

if not os.path.exists(base_path):
    os.makedirs(base_path)  # Create the directory 


# Process each file: Apply normalized areas and save the modified dataset


In [ ]:
# Process each file
for category, info in file_info.items():
    for period_key, file_path in info.items():
        if 'var_name' in period_key:
            continue  
        var_name = info[f'var_name_{period_key}']  

        # Load the original data
        ds = xr.open_dataset(file_path)
        data = ds[var_name]

        # Apply normalized areas
        weighted_data = data.values * normalized_areas  

        # Update the dataset
        ds[var_name] = (ds[var_name].dims, weighted_data)  

        # Define new file path
        new_file_name = f"{category}_{period_key}_weighted.nc"
        new_file_path = os.path.join(base_path, new_file_name)

        # Save the modified dataset 
        ds.to_netcdf(new_file_path)

        print(f'Processed and saved weighted data for {var_name} in {new_file_path}')
